In [61]:
import os

In [62]:
def make_filelist(path):
    import os

    file=[]
    for root, directories, files in os.walk(path, topdown=False):  
        for name in files:
            extens = os.path.splitext(name)[-1]
            if extens =='.fits':
                file.append(os.path.join(root,name))
        
    return file



In [63]:
def make_filelist_cat(path):
    import os

    file=[]
    for root, directories, files in os.walk(path, topdown=False):  
        for name in files:
            extens = os.path.splitext(name)[-1]
            if extens =='.cat':
                file.append(os.path.join(root,name))
        
    return file

In [64]:
def load_data(filelist):
# input = file list
# output = fits data 3D tensor (ex)2048x2048xN_data)
    import numpy as np
    from astropy.io.fits.hdu import hdulist
    from astropy.io import fits
    from astropy.io.fits import getdata
    from astropy.io.fits import getheader
    
    
    stacked_data = []
    for filename in filelist:
        hdulist = np.array(fits.getdata(filename,ext=0))
        hdulist = hdulist.byteswap().newbyteorder() # big endian -> little endian
        stacked_data.append(hdulist)
    return np.array(stacked_data)

In [65]:
def background(hdulist):
    import sep 
    back_hdulist=[]
   
    
    for i in aligned_hdulist:
        bkg=sep.Background(i, bw=256, bh=256)
        data_sub = data-bkg
        back_hdulist.append(data_sub)
        
    return back_hdulist

In [66]:
def backcheck(hdulist):
    import sep 
    back_hdulist=[]
   
    
    for i in hdulist:
        bkg=sep.Background(i, bw=256, bh=256)
        back_hdulist.append(bkg.globalrms)
        
    return back_hdulist


In [67]:
def sigmaclipping(hdulist):
    from astropy import stats
    
    clipped_list = []
    for i in hdulist:
        sigclip = stats.SigmaClip(sigma=2, maxiters=5)
        clipped_list.append(sigclip(i))
        
    return clipped_list

In [68]:
def sigmaclipping(hdulist):
    from astropy import stats
    
    clipped_list=[]
    for i in hdulist:
        sigclip = stats.SigmaClip(sigma=2, maxiters=5)
        clipped_list.append(sigclip(i))
        
    return clipped_list

In [490]:
def alignIMG(filelist, reference):
# input = filelist, reference image
# output = aligned images filelist, header
    import os
    from astropy.io import fits
    from astropy.io.fits import getdata
    from astropy.io.fits import getheader
    import astroalign as aa
      
    dir_ = os.path.dirname(filelist[0])
    hdulist = load_data(filelist)
    #hdulist=sigmaclipping(hdulist)
    

    reference_image = fits.getdata(reference,ext=0)
    reference_image = reference_image.byteswap().newbyteorder()
    reference_header = fits.getheader(reference,ext=0)
    os.chdir('/data3/yunyi/SOAO/select')
    os.system('rm -r aligned')
    os.mkdir('/data3/yunyi/SOAO/select/aligned')
    os.chdir('/data3/yunyi/SOAO/select/aligned')
    
    for i, filename in enumerate(filelist):
        file_name = os.path.split(filename)[1]
        aligned_image, footprint = aa.register(hdulist[i], reference_image)
        fits.writeto("aligned_%s"%file_name, aligned_image, reference_header)
    print('Reference image =', reference)

    print("Image saved directory:", os.getcwd())
    aligned_filelist = os.listdir(os.getcwd())
    return aligned_filelist, reference_header


In [279]:
def zeropoint(catloca, ali_selist):
    from astropy.coordinates import match_coordinates_sky
    from astropy.coordinates import SkyCoord
    from astropy.io import ascii
    import csv
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    
    ZP=[]
    for i in range(len(ali_selist)):
        catal=ascii.read(catloca)
        ra = catal['raMean']
        dec = catal['decMean']
        inflag = catal['objInfoFlag']
        qflag = catal['qualityFlag']
        c = SkyCoord(ra, dec, frame='icrs', unit='deg')

        ob = ascii.read(ali_selist[i])    
        ALPHA_J2000 = ob['ALPHA_J2000']
        DELTA_J2000 = ob['DELTA_J2000']
        flag = ob['FLAGS']
        o = SkyCoord(ALPHA_J2000, DELTA_J2000, frame='icrs', unit='deg')

        indx, sep, _ = o.match_to_catalog_sky(c)
        
        #print(sep.arcsec)
        P=[]
        sep_limit=2

        for i in range(len(o)):
            if sep.arcsec[i]<sep_limit:
                if flag[i]<1:
                    if inflag[i] < 10**9:
                        if qflag[i] < 100:
                            P.append(i)
                            
        mag_ob = np.array(ob['MAG_AUTO'])
        mag_catal = np.array(catal['R'])
        magErr = np.array(ob['MAGERR_AUTO'])
        gErr = np.array(catal['gMeanKronMagErr'])
        rErr = np.array(catal['rMeanKronMagErr'])
        FWHM = np.array(ob['FWHM_IMAGE'])

        #zero point & zero point error calculation 
        zp = []
        zpE = []
        for i in range(len(mag_ob)):
            u=indx[i]
            x = float(mag_catal[u])-float(mag_ob[i])
            e = np.sqrt(magErr[i]**2 + gErr[u]**2 + rErr[u]**2)
            zp.append(x)
            zpE.append(e)
            
        #zp versus ref mag
        mag_c_new=[]
        zp_new=[]
        zpE_new=[]
        seeing=[]
        for i in P:
            u=indx[i]
            if 27.5>zp[i]:
                if zpE[i]<0.1:
                    if 17>mag_catal[u]>14:
                        mag_c_new.append(mag_catal[u])
                        zpE_new.append(zpE[i])
                        zp_new.append(zp[i])
                        seeing.append(0.4454*FWHM[i])
        #print(zp_new)                
        ZP.append(np.mean(zp_new))
        print(np.mean(seeing))
        print(seeing)
                      
    return ZP

In [273]:
def scaling(catloca, aligned_filelist, ali_selist, reference_header, ZP):
    import os
    from astropy.io import fits
    from astropy.io.fits import getdata
    from astropy.io.fits import getheader
    import numpy as np
    #print(catloca)
    
    #ZP = zeropoint(catloca, ali_selist)
    #print(ZP)
    hdulist = load_data(aligned_filelist)
    os.chdir('/data3/yunyi/SOAO/select')
    os.system('rm -r scaled')
    os.mkdir('/data3/yunyi/SOAO/select/scaled')
    os.chdir('/data3/yunyi/SOAO/select/scaled')
    
    ZP_new =[]
    hdulist_new = []
    aligned_filelist_new = []
    for i in range(len(hdulist)):
        if np.isnan(ZP[i]) != True:
            ZP_new.append(ZP[i])
            hdulist_new.append(hdulist[i])
            aligned_filelist_new.append(aligned_filelist[i])
            
    ZPmax = max(ZP_new)

    ratio_flux = []
    for zp in ZP:
        dif = ZPmax - zp
        ratio = 10**(dif/2.5)
        ratio_flux.append(ratio)
    print(ratio_flux)
    
    scaled_image=[]
    for i, data in enumerate(hdulist_new):
        scaled_image.append(data*ratio_flux[i])
    
        
    for i, filename in enumerate(aligned_filelist_new):
        file_name = os.path.split(filename)[1]
        fits.writeto("scaled_%s"%file_name, scaled_image[i], reference_header)
    
    print("Image saved directory:", os.getcwd())
    scaled_filelist = os.listdir(os.getcwd())
    
    #reset directory iraf 
    path = '/data3/yunyi/SOAO/iraf'
    os.chdir('/data3/yunyi/SOAO/iraf')
    for root, directories, files in os.walk(path, topdown=False):  
        for name in files:
            extens = os.path.splitext(name)[-1]
            if extens =='.fits':
                os.system(f'rm {name}')
            elif extens =='.list':
                os.system(f'rm {name}')
                
    #move scaled files in iraf
    os.chdir('/data3/yunyi/SOAO/select/scaled')
    os.system('cp scaled_* /data3/yunyi/SOAO/iraf')
    os.chdir('/data3/yunyi/SOAO/iraf')
    os.system('ls scaled_*> new.list')
    
    return scaled_image

In [334]:
def make_selist(aligned_filelist, seeingval):
    #copy aligned files to SOAO
    os.chdir('/data3/yunyi/SOAO/select/aligned')
    os.system('cp aligned_Calib* /data3/yunyi/SOAO')
    
    #reset selist
    os.chdir('/data3/yunyi/SOAO/select')
    os.system('rm -r selist')
    os.mkdir('selist')
    
    #make new selist
    apt_pix = seeingval*2/0.4454
    os.chdir('/data3/yunyi/SOAO')
    for filename in aligned_filelist:
        os.system(f'sex {filename} -PHOT_APERTURES {apt_pix}')
        filename = filename[:-5]
        os.system(f'cp test.cat /data3/yunyi/SOAO/select/selist/{filename}.cat')
        
        
    #remove aligned files in SOAO
    os.chdir('/data3/yunyi/SOAO')
    os.system('rm aligned_Calib*')

In [476]:
filelist=make_filelist('/data3/yunyi/SOAO/select/NGC0628')
filelist=filelist[1:]
reference = filelist[0]

In [477]:
filelist

['/data3/yunyi/SOAO/select/NGC0628/Calib-SOAO-NGC0628-20191010-174424-R-60.fits',
 '/data3/yunyi/SOAO/select/NGC0628/Calib-SOAO-NGC0628-20191010-174529-R-60.fits',
 '/data3/yunyi/SOAO/select/NGC0628/Calib-SOAO-NGC0628-20191010-174634-R-60.fits',
 '/data3/yunyi/SOAO/select/NGC0628/Calib-SOAO-NGC0628-20191020-175426-R-60.fits',
 '/data3/yunyi/SOAO/select/NGC0628/Calib-SOAO-NGC0628-20201111-105920-R-60.fits',
 '/data3/yunyi/SOAO/select/NGC0628/Calib-SOAO-NGC0628-20201111-110235-R-60.fits',
 '/data3/yunyi/SOAO/select/NGC0628/Calib-SOAO-NGC0628-20201111-110340-R-60.fits',
 '/data3/yunyi/SOAO/select/NGC0628/Calib-SOAO-NGC0628-20201112-115818-R-60.fits',
 '/data3/yunyi/SOAO/select/NGC0628/Calib-SOAO-NGC0628-20201112-120133-R-60.fits']

In [478]:
aligned_filelist, reference_header=alignIMG(filelist, reference)


Reference image = /data3/yunyi/SOAO/select/NGC0628/Calib-SOAO-NGC0628-20191010-174424-R-60.fits
Image saved directory: /data3/yunyi/SOAO/select/aligned


In [479]:
print(aligned_filelist)
print(reference_header)

['aligned_Calib-SOAO-NGC0628-20191010-174424-R-60.fits', 'aligned_Calib-SOAO-NGC0628-20191010-174529-R-60.fits', 'aligned_Calib-SOAO-NGC0628-20191010-174634-R-60.fits', 'aligned_Calib-SOAO-NGC0628-20191020-175426-R-60.fits', 'aligned_Calib-SOAO-NGC0628-20201111-105920-R-60.fits', 'aligned_Calib-SOAO-NGC0628-20201111-110235-R-60.fits', 'aligned_Calib-SOAO-NGC0628-20201111-110340-R-60.fits', 'aligned_Calib-SOAO-NGC0628-20201112-115818-R-60.fits', 'aligned_Calib-SOAO-NGC0628-20201112-120133-R-60.fits']
SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                 2048                                                  NAXIS2  =                 2048                                                  EXTEND  =                    T                                                  OBSERVAT= 'SOAO

In [480]:
aligned_filelist_pathadd=make_filelist('/data3/yunyi/SOAO/select/aligned')
print(aligned_filelist_pathadd)

['/data3/yunyi/SOAO/select/aligned/aligned_Calib-SOAO-NGC0628-20191010-174424-R-60.fits', '/data3/yunyi/SOAO/select/aligned/aligned_Calib-SOAO-NGC0628-20191010-174529-R-60.fits', '/data3/yunyi/SOAO/select/aligned/aligned_Calib-SOAO-NGC0628-20191010-174634-R-60.fits', '/data3/yunyi/SOAO/select/aligned/aligned_Calib-SOAO-NGC0628-20191020-175426-R-60.fits', '/data3/yunyi/SOAO/select/aligned/aligned_Calib-SOAO-NGC0628-20201111-105920-R-60.fits', '/data3/yunyi/SOAO/select/aligned/aligned_Calib-SOAO-NGC0628-20201111-110235-R-60.fits', '/data3/yunyi/SOAO/select/aligned/aligned_Calib-SOAO-NGC0628-20201111-110340-R-60.fits', '/data3/yunyi/SOAO/select/aligned/aligned_Calib-SOAO-NGC0628-20201112-115818-R-60.fits', '/data3/yunyi/SOAO/select/aligned/aligned_Calib-SOAO-NGC0628-20201112-120133-R-60.fits']


In [481]:
make_selist(aligned_filelist, 1.8)

In [482]:
ali_selist=make_filelist_cat('/data3/yunyi/SOAO/select/selist')

In [483]:
ali_selist

['/data3/yunyi/SOAO/select/selist/aligned_Calib-SOAO-NGC0628-20191010-174424-R-60.cat',
 '/data3/yunyi/SOAO/select/selist/aligned_Calib-SOAO-NGC0628-20191010-174529-R-60.cat',
 '/data3/yunyi/SOAO/select/selist/aligned_Calib-SOAO-NGC0628-20191010-174634-R-60.cat',
 '/data3/yunyi/SOAO/select/selist/aligned_Calib-SOAO-NGC0628-20191020-175426-R-60.cat',
 '/data3/yunyi/SOAO/select/selist/aligned_Calib-SOAO-NGC0628-20201111-105920-R-60.cat',
 '/data3/yunyi/SOAO/select/selist/aligned_Calib-SOAO-NGC0628-20201111-110235-R-60.cat',
 '/data3/yunyi/SOAO/select/selist/aligned_Calib-SOAO-NGC0628-20201111-110340-R-60.cat',
 '/data3/yunyi/SOAO/select/selist/aligned_Calib-SOAO-NGC0628-20201112-115818-R-60.cat',
 '/data3/yunyi/SOAO/select/selist/aligned_Calib-SOAO-NGC0628-20201112-120133-R-60.cat']

In [484]:
catloca='/data3/yunyi/SOAO/code/NGC0628_R.csv'
ZP=zeropoint(catloca, ali_selist)

1.969304285714286
[1.62571, 1.73706, 1.5856240000000001, 1.576716, 1.5099060000000002, 1.674704, 1.9820300000000002, 1.719244, 1.607894, 1.5856240000000001, 1.5544460000000002, 1.5544460000000002, 1.567808, 1.674704, 1.621256, 1.6791580000000002, 4.3738280000000005, 1.4653660000000002, 5.839194, 1.786054, 1.6346180000000001]
2.1457145
[1.754876, 2.11565, 1.598986, 1.6346180000000001, 1.6568880000000001, 1.7058820000000001, 1.9107660000000002, 1.973122, 1.69252, 1.652434, 1.621256, 1.6346180000000001, 1.6836119999999999, 1.701428, 1.7816, 4.280294, 1.73706, 7.130854000000001, 1.9775760000000002, 1.67025]
2.148100571428571
[1.7281520000000001, 1.852864, 1.754876, 1.754876, 1.7504220000000001, 1.75933, 1.8662260000000002, 2.053294, 1.835048, 1.710336, 1.69252, 1.7771460000000001, 1.835048, 1.763784, 1.790508, 4.360466, 1.8305940000000003, 1.7504220000000001, 6.066348, 2.396252, 1.7816]
2.287235047619048
[1.7058820000000001, 1.8305940000000003, 2.0043, 1.8884960000000002, 1.705882000000000

In [485]:
ZP

[25.749642238095237,
 25.744313950000002,
 25.749839561904757,
 25.58815332380953,
 25.633227090000002,
 25.62636174166667,
 25.64613903809524,
 25.681163622222222,
 25.645963123809526]

In [486]:
scaled_image=scaling(catloca, aligned_filelist_pathadd, ali_selist, reference_header,ZP)

[1.000181758460985, 1.0051022490063022, 1.0, 1.160578431444948, 1.11338403286598, 1.1204465097490353, 1.100221707869308, 1.0652961951271236, 1.1003999835823364]
Image saved directory: /data3/yunyi/SOAO/select/scaled


In [487]:
scaled_image[1]

array([[ 99.36481048,  74.47532248,  89.91531184, ...,  83.43106491,
        115.1995644 ,  86.22023261],
       [108.76621136,  86.57080072,  77.38112721, ...,  93.91907225,
        131.44615225,  85.69986165],
       [ 94.07713454,  80.61941876,  73.85656112, ...,  99.59781986,
        104.49593153,  86.54056959],
       ...,
       [ 69.91284544,  78.41064651,  82.37248046, ...,  85.61181958,
        117.56339811,  85.60230186],
       [ 70.12908402,  82.97350721,  83.23983272, ..., 118.89092207,
         82.75654648,  87.23453988],
       [ 80.3187123 ,  93.1786659 ,  74.03849721, ...,  77.31554508,
         82.49802977,  87.35086356]])

In [489]:

#after combine

file='NGC0628_median.fits'
file2 = 'NGC0628_median.cat'
os.chdir('/data3/yunyi/SOAO/iraf')
os.system(f'cp {file} /data3/yunyi/SOAO' )


os.chdir('/data3/yunyi/SOAO')
seeingval = 1.9
apt_pix = seeingval*2/0.4454
os.system(f'sex {file} -PHOT_APERTURES {apt_pix}')
os.system(f'cp test.cat ./code/{file2}')


0

In [373]:
from astropy.io import ascii
import numpy as np


seeingval = 2.2
apt_pix = seeingval*2/0.4454

os.chdir('/data3/yunyi/SOAO')
#apt_pix = np.mean(seeing)
os.system(f'sex {file} -PHOT_APERTURES {apt_pix}')
os.system(f'cp test.cat ./code/{file2}')


0

In [313]:
seeingval

2

In [ ]:
'''
filelist=['NGC0514_median.fits', 'M99_mean.fits']
filelist2 = ['M99_median.cat', 'M99_mean.cat']
os.chdir('/data3/yunyi/SOAO/iraf')
os.system(f'cp {filelist[0]} /data3/yunyi/SOAO' )
os.system(f'cp {filelist[1]} /data3/yunyi/SOAO')

os.chdir('/data3/yunyi/SOAO')
for i in range(2):
    os.system(f'sex {filelist[i]}')
    os.system(f'cp test.cat ./code/{filelist2[i]}')

from astropy.io import ascii
import numpy as np

#obj = ascii.read('/data3/yunyi/SOAO/code/M51A_median.cat') 
#FWHM = np.array(obj['FWHM_IMAGE'])
#seeing=0.4454*FWHM
#print(seeing)
seeingval = 
apt_pix = seeingval*2/0.4454

os.chdir('/data3/yunyi/SOAO')
apt_pix = np.mean(seeing)
for i in range(2):
    os.system(f'sex {filelist[i]} -PHOT_APERTURES {apt_pix}')
    os.system(f'cp test.cat ./code/{filelist2[i]}')
    
'''